### I. Data Importing and Initial Cleaning
I approached the table creation and five questions somewhat out of order (for example, answering Question 1 last). Please see headings and markdown for notes on the work for each of the five questions. 

In [1]:
#Import all libraries for data cleaning and merging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from thefuzz import process
from fuzzywuzzy import fuzz

#Pandas settings
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',500)
pd.options.display.float_format = "{:,.2f}".format

In [2]:
#Read in all multistat datafiles
#I have already moved the site tabs to the front & simplified column headings in Excel 
ms19 = pd.read_excel('data/2019-ms.xlsx')
ms21 = pd.read_excel('data/2021-ms.xlsx')
ms22 = pd.read_excel('data/2022-ms.xlsx')

#Read in all LEAP data
leap19 = pd.read_excel('data/2019-leap.xlsx')
leap21 = pd.read_excel('data/2021-leap.xlsx')
leap22 = pd.read_excel('data/2022-leap.xlsx')

In [3]:
#Dealing with multistats first
#Preview size of all files
print(ms19.shape)
print(ms21.shape)
print(ms22.shape)

(1414, 40)
(1399, 41)
(1394, 41)


In [4]:
ms19.head()

,School System,School System Name,SiteCd,FedSiteCd,SiteName,Total Students,%Female,%Male,AmInd,Asian,Black,Hispanic,HawPI,White,Multiple,Minority,%Fully-EP,%LEP,Infants SpEd,PreSchool SpEd,PreK,Kindergarten,Grade1,Grade2,Grade3,Grade4,Grade5,Grade6,Grade7,Grade8,Grade9,GradeT9,Grade10,Grade11,Grade12,ED%,Nonprofit,Charter Type,RollUpType,Parish Code
0,001,Acadia Parish,001001,001001,Armstrong Middle School,341,0.42,0.58,0,1,146,12,0,166,16,175,0.99,0.01,0,0,0,0,0,0,0,0,0,112,125,104,0,0,0,0,0,0.79,NaN,NaN,NaN,1.00
1,001,Acadia Parish,001002,001002,Branch Elementary School,277,0.49,0.51,0,0,20,9,0,240,8,37,0.99,0.01,0,1,19,35,28,31,31,35,25,24,27,21,0,0,0,0,0,0.55,NaN,NaN,NaN,1.00
2,001,Acadia Parish,001003,001003,Central Rayne Kindergarten School,215,0.51,0.49,0,0,85,1,0,111,18,104,1.00,0.00,0,12,68,135,0,0,0,0,0,0,0,0,0,0,0,0,0,0.79,NaN,NaN,NaN,1.00
3,001,Acadia Parish,001004,001004,Church Point Elementary School,587,0.46,0.54,4,0,237,47,0,243,56,344,0.99,0.01,0,3,55,88,79,91,91,90,90,0,0,0,0,0,0,0,0,0.85,NaN,NaN,NaN,1.00
4,001,Acadia Parish,001005,001005,Church Point High School,514,0.48,0.52,2,0,151,19,0,317,25,197,1.00,0.00,0,0,0,0,0,0,0,0,0,0,0,0,125,27,137,127,98,0.61,NaN,NaN,NaN,1.00


In [5]:
#Grabbing all variations of Orleans Parish before I filter
ms19['School System Name'].value_counts()

East Baton Rouge Parish                          86
Jefferson Parish                                 85
All Orleans                                      79
Caddo Parish                                     64
Calcasieu Parish                                 60
St. Tammany Parish                               56
Rapides Parish                                   47
Lafayette Parish                                 44
Livingston Parish                                44
Type 2 Charters                                  43
Ouachita Parish                                  37
Tangipahoa Parish                                35
St. Landry Parish                                34
Bossier Parish                                   34
Terrebonne Parish                                33
Acadia Parish                                    32
Lafourche Parish                                 31
Ascension Parish                                 28
Iberia Parish                                    26
St. Mary Par

In [6]:
ms19.loc[ms19['School System Name'] == 'RSD-Direct Run']

,School System,School System Name,SiteCd,FedSiteCd,SiteName,Total Students,%Female,%Male,AmInd,Asian,Black,Hispanic,HawPI,White,Multiple,Minority,%Fully-EP,%LEP,Infants SpEd,PreSchool SpEd,PreK,Kindergarten,Grade1,Grade2,Grade3,Grade4,Grade5,Grade6,Grade7,Grade8,Grade9,GradeT9,Grade10,Grade11,Grade12,ED%,Nonprofit,Charter Type,RollUpType,Parish Code
1337,RLA,RSD-Direct Run,396211,396211,Linwood Charter School,980,0.54,0.46,0,0,949,19,0,7,5,973,0.99,0.01,0,0,20,122,120,109,120,115,72,104,99,99,0,0,0,0,0,0.97,NaN,NaN,RLA,9.00


In [7]:
#Ignoring Linwood as it is in Shreveport and ignoring NOCCA as it is a Type 2 Charter
#Let's pull Orleans parish schools only
#Starting with SY19 
nolams19 = ms19.loc[(ms19['School System Name'] == 'All Orleans')]

In [8]:
nolams19.shape

(79, 40)

In [9]:
#Repeating process for SY21 ms file
ms21.head()

,School System,School System Name,SIS Submit Site Code,Federal Reporting Site Code,Site Name,Total Enrollment,% Female,% Male,American Indian,Asian,Black,Hispanic,Hawaiian/Pacific Islander,White,Multiple Races (Non-Hispanic),Minority,% Fully English Proficient,% Limited English Proficient,Infants (Sp Ed),Pre-School (Sp Ed),Pre-K (Reg Ed),Kindergarten,Grade 1,Grade 2,Grade 3,Grade 4,Grade 5,Grade 6,Grade 7,Grade 8,Grade 9,Grade T9,Grade 10,Grade 11,Grade 12,Extension Academy,% Economically Disadvantaged,Nonprofit Organization,Charter Type,School System Roll Up Type,Parish Code
0,School System,School System Name,SiteCd,FedSiteCd,SiteName,Total Students,%Female,%Male,AmInd,Asian,Black,Hispanic,HawPI,White,Multiple,Minority,%Fully-EP,%LEP,Infants SpEd,PreSchool SpEd,PreK,Kindergarten,Grade1,Grade2,Grade3,Grade4,Grade5,Grade6,Grade7,Grade8,Grade9,GradeT9,Grade10,Grade11,Grade12,Extension Academy,ED%,Nonprofit,Charter Type,RollUpType,Parish Code
1,001,Acadia Parish,001001,001001,Armstrong Middle School,337,0.52,0.48,1,0,157,9,0,145,25,192,0.99,0.01,0,0,0,0,0,0,0,0,0,131,96,110,0,0,0,0,0,0,0.88,NaN,NaN,NaN,01
2,001,Acadia Parish,001002,001002,Branch Elementary School,348,0.49,0.51,0,1,28,16,0,290,13,58,0.99,0.01,0,1,19,33,48,39,39,39,30,40,37,23,0,0,0,0,0,0,0.60,NaN,NaN,NaN,01
3,001,Acadia Parish,001003,001003,Central Rayne Kindergarten School,196,0.48,0.52,2,1,78,7,0,101,7,95,0.99,0.01,0,6,56,134,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.83,NaN,NaN,NaN,01
4,001,Acadia Parish,001004,001004,Church Point Elementary School,577,0.47,0.53,3,0,248,48,0,240,38,337,0.98,0.02,0,4,58,90,96,85,91,79,74,0,0,0,0,0,0,0,0,0,0.91,NaN,NaN,NaN,01


In [10]:
ms21['School System Name'].value_counts()

East Baton Rouge Parish                          82
Jefferson Parish                                 81
Orleans Parish                                   77
Calcasieu Parish                                 61
Caddo Parish                                     59
St. Tammany Parish                               57
Rapides Parish                                   51
Lafayette Parish                                 46
Livingston Parish                                44
Type 2 Charters                                  39
Ouachita Parish                                  37
Bossier Parish                                   35
Tangipahoa Parish                                34
St. Landry Parish                                33
Lafourche Parish                                 32
Acadia Parish                                    32
Terrebonne Parish                                32
Ascension Parish                                 31
Iberia Parish                                    24
St. Mary Par

In [11]:
ms21.loc[ms21['School System Name'] == 'Recovery School District - Louisiana']

,School System,School System Name,SIS Submit Site Code,Federal Reporting Site Code,Site Name,Total Enrollment,% Female,% Male,American Indian,Asian,Black,Hispanic,Hawaiian/Pacific Islander,White,Multiple Races (Non-Hispanic),Minority,% Fully English Proficient,% Limited English Proficient,Infants (Sp Ed),Pre-School (Sp Ed),Pre-K (Reg Ed),Kindergarten,Grade 1,Grade 2,Grade 3,Grade 4,Grade 5,Grade 6,Grade 7,Grade 8,Grade 9,Grade T9,Grade 10,Grade 11,Grade 12,Extension Academy,% Economically Disadvantaged,Nonprofit Organization,Charter Type,School System Roll Up Type,Parish Code
1397,RLA,Recovery School District - Louisiana,396211,396211,Linwood Charter School,965,0.52,0.48,0,0,914,39,0,3,9,962,0.97,0.03,0,0,30,95,108,103,110,99,103,122,103,92,0,0,0,0,0,0,0.97,NaN,NaN,RLA,09


In [12]:
nolams21 = ms21.loc[ms21['School System Name'] == 'Orleans Parish']

In [13]:
#One more time for SY 22 multistat file
ms22.head()

,School System,School System Name,SIS Submit Site Code,Federal Reporting Site Code,Site Name,Total Enrollment,% Female,% Male,American Indian,Asian,Black,Hispanic,Hawaiian/Pacific Islander,White,Multiple Races (Non-Hispanic),Minority,% Fully English Proficient,% Limited English Proficient,Infants (Sp Ed),Pre-School (Sp Ed),Pre-K (Reg Ed),Kindergarten,Grade 1,Grade 2,Grade 3,Grade 4,Grade 5,Grade 6,Grade 7,Grade 8,Grade 9,Grade T9,Grade 10,Grade 11,Grade 12,Extension Academy,% Economically Disadvantaged,Nonprofit Organization,Charter Type,School System Roll Up Type,Parish Code
0,School System,School System Name,SiteCd,FedSiteCd,SiteName,Total Students,%Female,%Male,AmInd,Asian,Black,Hispanic,HawPI,White,Multiple,Minority,%Fully-EP,%LEP,Infants SpEd,PreSchool SpEd,PreK,Kindergarten,Grade1,Grade2,Grade3,Grade4,Grade5,Grade6,Grade7,Grade8,Grade9,GradeT9,Grade10,Grade11,Grade12,Extension Academy,ED%,Nonprofit,Charter Type,RollUpType,Parish Code
1,001,Acadia Parish,001001,001001,Armstrong Middle School,338,0.46,0.54,0,1,160,10,0,143,24,195,1.00,0.00,0,0,0,0,0,0,0,0,0,129,117,92,0,0,0,0,0,0,0.83,NaN,NaN,NaN,01
2,001,Acadia Parish,001002,001002,Branch Elementary School,353,0.47,0.53,0,1,39,14,0,288,11,65,0.99,0.01,0,3,17,36,36,48,38,39,39,32,36,29,0,0,0,0,0,0,0.56,NaN,NaN,NaN,01
3,001,Acadia Parish,001003,001003,Central Rayne Kindergarten School,204,0.51,0.49,3,1,82,8,0,95,15,109,0.99,0.01,0,5,71,128,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.81,NaN,NaN,NaN,01
4,001,Acadia Parish,001004,001004,Church Point Elementary School,601,0.45,0.55,3,1,255,51,0,250,41,351,0.97,0.03,0,6,54,104,91,91,84,88,83,0,0,0,0,0,0,0,0,0,0.85,NaN,NaN,NaN,01


In [14]:
ms22['School System Name'].value_counts()

East Baton Rouge Parish                          84
Jefferson Parish                                 82
Orleans Parish                                   77
Calcasieu Parish                                 60
Caddo Parish                                     59
St. Tammany Parish                               56
Rapides Parish                                   48
Lafayette Parish                                 46
Livingston Parish                                44
Type 2 Charters                                  40
Ouachita Parish                                  37
Bossier Parish                                   35
Tangipahoa Parish                                34
St. Landry Parish                                34
Lafourche Parish                                 32
Ascension Parish                                 32
Acadia Parish                                    32
Terrebonne Parish                                31
Iberia Parish                                    24
St. Mary Par

In [15]:
nolams22 = ms22.loc[ms22['School System Name'] == 'Orleans Parish']

In [16]:
#PLAN - 
#Merge LEAP and multistat files for each year (2019, 2021, 2022) -> 3 separate files
#Add years to each column heading
#Merge datafiles together, using an outer join

print(nolams19.shape)
print(nolams21.shape)
print(nolams22.shape)

(79, 40)
(77, 41)
(77, 41)


In [17]:
nolams19.head(79)

,School System,School System Name,SiteCd,FedSiteCd,SiteName,Total Students,%Female,%Male,AmInd,Asian,Black,Hispanic,HawPI,White,Multiple,Minority,%Fully-EP,%LEP,Infants SpEd,PreSchool SpEd,PreK,Kindergarten,Grade1,Grade2,Grade3,Grade4,Grade5,Grade6,Grade7,Grade8,Grade9,GradeT9,Grade10,Grade11,Grade12,ED%,Nonprofit,Charter Type,RollUpType,Parish Code
746,R36,All Orleans,036011,036011,Mary Bethune Elementary Literature/Technology,688,0.52,0.48,0,2,669,15,0,2,0,686,0.99,0.01,0,2,38,78,78,80,81,79,82,54,55,61,0,0,0,0,0,0.82,NaN,NaN,R36,36.00
747,R36,All Orleans,036035,036035,Warren Easton Senior High School,991,0.51,0.49,0,0,960,25,0,1,5,990,0.98,0.02,0,0,0,0,0,0,0,0,0,0,0,0,229,15,247,245,255,0.78,"Warren Easton Senior High School Foundation, Inc.",Type 3,R36,36.00
748,R36,All Orleans,036060,036060,Edward Hynes Charter School,717,0.53,0.47,0,32,252,30,0,361,42,356,0.97,0.03,0,17,0,83,84,79,79,72,80,77,76,70,0,0,0,0,0,0.35,Hynes Charter School Corporation,Type 3,R36,36.00
749,R36,All Orleans,036088,036088,McDonogh #35 College Preparatory School,445,0.44,0.56,0,1,442,2,0,0,0,445,1.00,0.00,0,0,0,0,0,0,0,0,0,0,0,0,10,0,146,107,182,0.89,NaN,NaN,R36,36.00
750,R36,All Orleans,036132,036132,Youth Study Center,106,0.06,0.94,0,0,104,2,0,0,0,106,0.99,0.01,0,0,0,0,0,0,0,0,0,0,0,4,35,0,32,14,21,0.90,NaN,NaN,R36,36.00
751,R36,All Orleans,036161,036161,Benjamin Franklin Elem. Math and Science,797,0.49,0.51,2,7,742,13,2,24,7,773,0.98,0.02,0,26,16,74,78,82,82,83,88,93,87,88,0,0,0,0,0,0.75,NaN,NaN,R36,36.00
752,R36,All Orleans,036189,036189,Homer A. Plessy Community School,397,0.48,0.52,2,7,177,48,0,118,45,279,0.92,0.08,0,0,39,50,42,46,44,54,53,40,29,0,0,0,0,0,0,0.78,Citizens' Committee for Education,Type 1,R36,36.00
753,R36,All Orleans,036192,036192,Foundation Preparatory,177,0.44,0.56,0,5,125,41,2,4,0,173,0.73,0.27,0,0,0,32,53,42,23,27,0,0,0,0,0,0,0,0,0,0.99,Foundation Prep,Type 1,R36,36.00
754,R36,All Orleans,036193,036193,Cypress Academy,184,0.43,0.57,0,1,93,26,0,56,8,128,0.95,0.05,0,0,0,41,34,36,45,28,0,0,0,0,0,0,0,0,0,0.68,Cypress Academy,Type 1,R36,36.00
755,R36,All Orleans,036197,036197,Elan Academy Charter School,143,0.47,0.53,0,1,116,6,0,11,9,132,0.97,0.03,0,0,18,33,48,22,22,0,0,0,0,0,0,0,0,0,0,0.78,"Elan Academy, Inc.",Type 1,R36,36.00


In [18]:
#Next, dealing with LEAP files, starting with SY19
leap19.head()

,School System Code,School System Name,Site Code,Site Name,2018 % Mastery+ Grades 3-8,2019 % Mastery+ Grades 3-8,2018-2019 % Mastery+ Change Grades 3-8,2018 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History,2019 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History,2018-2019 % Mastery+ Change Grades 3-8 & English I-II & Algebra I & Geometry & US History
0,001,Acadia Parish,001001,Armstrong Middle School,18,27,9,18,27,9
1,001,Acadia Parish,001002,Branch Elementary School,54,54,0,54,54,0
2,001,Acadia Parish,001004,Church Point Elementary School,32,28,-4,32,28,-4
3,001,Acadia Parish,001005,Church Point High School,NR,NR,NR,31,26,-5
4,001,Acadia Parish,001006,Church Point Middle School,22,27,5,22,27,5


In [19]:
leap19['School System Name'].value_counts()

Jefferson Parish                                      84
East Baton Rouge Parish                               82
Orleans Parish                                        76
Caddo Parish                                          62
Calcasieu Parish                                      53
St. Tammany Parish                                    50
Rapides Parish                                        42
Lafayette Parish                                      42
Livingston Parish                                     42
Ouachita Parish                                       33
St. Landry Parish                                     33
Tangipahoa Parish                                     32
Terrebonne Parish                                     31
Bossier Parish                                        31
New Orleans Archdiocese                               30
Ascension Parish                                      26
Lafourche Parish                                      26
Acadia Parish                  

In [20]:
#Lycée Français and other schools at the bottom of the list seem to be Type 2 charters
#Pulling only schools labeled as "Orleans Parish" 
nolaleap19 = leap19.loc[leap19['School System Name'] == 'Orleans Parish']

In [21]:
print(nolams19.shape)
print(nolaleap19.shape)
#Seems there are three schools that either closed or did not report LEAP scores in 2019

(79, 40)
(76, 10)


In [22]:
#Joining SY19 dataframes 
#Using an outer join to preserve those 3 schools that did not report LEAP data
nolasy19 = nolams19.merge(nolaleap19, left_on=['SiteCd'], right_on=['Site Code'], how='outer', indicator=True)

In [23]:
nolasy19.shape
#Join seems to have worked as it preserves the three schools

(79, 51)

In [24]:
nolasy19.loc[nolasy19['_merge'] == 'left_only']
#Orleans Central Office, Joseph S. Clark, and Audobon Charter didn't report LEAP data for various reasons
#e.g., Central Office seems to be PreK only, but
#we'll still keep them in the final dataframe

,School System,School System Name_x,SiteCd,FedSiteCd,SiteName,Total Students,%Female,%Male,AmInd,Asian,Black,Hispanic,HawPI,White,Multiple,Minority,%Fully-EP,%LEP,Infants SpEd,PreSchool SpEd,PreK,Kindergarten,Grade1,Grade2,Grade3,Grade4,Grade5,Grade6,Grade7,Grade8,Grade9,GradeT9,Grade10,Grade11,Grade12,ED%,Nonprofit,Charter Type,RollUpType,Parish Code,School System Code,School System Name_y,Site Code,Site Name,2018 % Mastery+ Grades 3-8,2019 % Mastery+ Grades 3-8,2018-2019 % Mastery+ Change Grades 3-8,2018 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History,2019 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History,2018-2019 % Mastery+ Change Grades 3-8 & English I-II & Algebra I & Geometry & US History,_merge
11,R36,All Orleans,036700,036700,Orleans Central Office,269,0.39,0.61,0,8,187,12,0,53,9,216,1.00,0.00,0,269,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.62,NaN,NaN,R36,36.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
52,R36,All Orleans,399003,W93001,Joseph S. Clark Preparatory High School,43,0.44,0.56,0,0,41,2,0,0,0,43,0.98,0.02,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,42,0.84,"FirstLine Schools, Inc.",Type 3B,R36,36.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
76,R36,All Orleans,WBT001,WBT001,Audubon Charter School - Gentilly,151,0.59,0.41,0,5,110,6,0,23,7,128,0.99,0.01,0,12,22,36,39,42,0,0,0,0,0,0,0,0,0,0,0,0.66,"French and Montessori Education, Inc.",Type 1,R36,36.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [25]:
nolasy19['SiteCd'].isna().sum()

0

In [26]:
nolasy19['SiteName'].isna().sum()

0

In [27]:
nolasy19.loc[nolasy19['_merge'] == 'right_only']
#Luckily, it seems all schools who reported LEAP scores were also present in the multistat file
#We can be confident in dropping the "Site Code" and "Site Name" columns from the dataframe 

,School System,School System Name_x,SiteCd,FedSiteCd,SiteName,Total Students,%Female,%Male,AmInd,Asian,Black,Hispanic,HawPI,White,Multiple,Minority,%Fully-EP,%LEP,Infants SpEd,PreSchool SpEd,PreK,Kindergarten,Grade1,Grade2,Grade3,Grade4,Grade5,Grade6,Grade7,Grade8,Grade9,GradeT9,Grade10,Grade11,Grade12,ED%,Nonprofit,Charter Type,RollUpType,Parish Code,School System Code,School System Name_y,Site Code,Site Name,2018 % Mastery+ Grades 3-8,2019 % Mastery+ Grades 3-8,2018-2019 % Mastery+ Change Grades 3-8,2018 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History,2019 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History,2018-2019 % Mastery+ Change Grades 3-8 & English I-II & Algebra I & Geometry & US History,_merge


In [28]:
#Cleaning up SY19 columns
#Dropping redundant columns - FedSiteCd, School System, School System Name_x and School System Name_y 
#(they are all Orleans)
#Alsop dropping _merge, Parish Code, and School System Code, Site Code, and Site Name as per the check above 
nolasy19 = nolasy19.drop(columns=['School System','School System Name_x', 'FedSiteCd','School System Name_y',
                                '_merge','Parish Code','School System Code','Site Code','Site Name'], axis=1)


In [29]:
nolasy19.shape

(79, 42)

In [30]:
nolasy19.head(1)

,SiteCd,SiteName,Total Students,%Female,%Male,AmInd,Asian,Black,Hispanic,HawPI,White,Multiple,Minority,%Fully-EP,%LEP,Infants SpEd,PreSchool SpEd,PreK,Kindergarten,Grade1,Grade2,Grade3,Grade4,Grade5,Grade6,Grade7,Grade8,Grade9,GradeT9,Grade10,Grade11,Grade12,ED%,Nonprofit,Charter Type,RollUpType,2018 % Mastery+ Grades 3-8,2019 % Mastery+ Grades 3-8,2018-2019 % Mastery+ Change Grades 3-8,2018 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History,2019 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History,2018-2019 % Mastery+ Change Grades 3-8 & English I-II & Algebra I & Geometry & US History
0,036011,Mary Bethune Elementary Literature/Technology,688,0.52,0.48,0,2,669,15,0,2,0,686,0.99,0.01,0,2,38,78,78,80,81,79,82,54,55,61,0,0,0,0,0,0.82,NaN,NaN,R36,31,33,2,31,34,3


In [31]:
#Add 'SY19' to all column names 
nolasy19 = nolasy19.add_prefix('sy19_')

In [32]:
nolasy19.head()

,sy19_SiteCd,sy19_SiteName,sy19_Total Students,sy19_%Female,sy19_%Male,sy19_AmInd,sy19_Asian,sy19_Black,sy19_Hispanic,sy19_HawPI,sy19_White,sy19_Multiple,sy19_Minority,sy19_%Fully-EP,sy19_%LEP,sy19_Infants SpEd,sy19_PreSchool SpEd,sy19_PreK,sy19_Kindergarten,sy19_Grade1,sy19_Grade2,sy19_Grade3,sy19_Grade4,sy19_Grade5,sy19_Grade6,sy19_Grade7,sy19_Grade8,sy19_Grade9,sy19_GradeT9,sy19_Grade10,sy19_Grade11,sy19_Grade12,sy19_ED%,sy19_Nonprofit,sy19_Charter Type,sy19_RollUpType,sy19_2018 % Mastery+ Grades 3-8,sy19_2019 % Mastery+ Grades 3-8,sy19_2018-2019 % Mastery+ Change Grades 3-8,sy19_2018 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History,sy19_2019 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History,sy19_2018-2019 % Mastery+ Change Grades 3-8 & English I-II & Algebra I & Geometry & US History
0,036011,Mary Bethune Elementary Literature/Technology,688,0.52,0.48,0,2,669,15,0,2,0,686,0.99,0.01,0,2,38,78,78,80,81,79,82,54,55,61,0,0,0,0,0,0.82,NaN,NaN,R36,31,33,2,31,34,3
1,036035,Warren Easton Senior High School,991,0.51,0.49,0,0,960,25,0,1,5,990,0.98,0.02,0,0,0,0,0,0,0,0,0,0,0,0,229,15,247,245,255,0.78,"Warren Easton Senior High School Foundation, Inc.",Type 3,R36,NR,NR,NR,28,21,-7
2,036060,Edward Hynes Charter School,717,0.53,0.47,0,32,252,30,0,361,42,356,0.97,0.03,0,17,0,83,84,79,79,72,80,77,76,70,0,0,0,0,0,0.35,Hynes Charter School Corporation,Type 3,R36,61,63,2,60,63,3
3,036088,McDonogh #35 College Preparatory School,445,0.44,0.56,0,1,442,2,0,0,0,445,1.00,0.00,0,0,0,0,0,0,0,0,0,0,0,0,10,0,146,107,182,0.89,NaN,NaN,R36,NR,NR,NR,9,8,-1
4,036132,Youth Study Center,106,0.06,0.94,0,0,104,2,0,0,0,106,0.99,0.01,0,0,0,0,0,0,0,0,0,0,0,4,35,0,32,14,21,0.90,NaN,NaN,R36,2,≤ 1,>-5,2,≤ 1,>-5


In [33]:
#Repeat LEAP filtering, merge and column cleaning process for the other three years 
leap21.head()

,School System Code,School System Name,Site Code,Site Name,2019 % Mastery+ Grades 3-8,2021 % Mastery+ Grades 3-8,2019-2021 % Mastery+ Change Grades 3-8,2019 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History & Biology,2021 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History & Biology,2019-2021 % Mastery+ Change Grades 3-8 & English I-II & Algebra I & Geometry & US History & Biology
0,001,Acadia Parish,001001,Armstrong Middle School,26,23,-3,26,23,-3
1,001,Acadia Parish,001002,Branch Elementary School,53,44,-9,53,44,-9
2,001,Acadia Parish,001004,Church Point Elementary School,28,31,3,28,31,3
3,001,Acadia Parish,001005,Church Point High School,NR,NR,NR,24,23,-1
4,001,Acadia Parish,001006,Church Point Middle School,24,22,-2,24,22,-2


In [34]:
leap21['School System Name'].value_counts()

Jefferson Parish                                      80
East Baton Rouge Parish                               77
Orleans Parish                                        74
Caddo Parish                                          57
Calcasieu Parish                                      52
St. Tammany Parish                                    51
Lafayette Parish                                      44
Rapides Parish                                        42
Livingston Parish                                     42
Ouachita Parish                                       33
Bossier Parish                                        32
St. Landry Parish                                     32
Terrebonne Parish                                     31
Tangipahoa Parish                                     31
Ascension Parish                                      29
Lafourche Parish                                      26
New Orleans Archdiocese                               25
Acadia Parish                  

In [35]:
#Again, since Lycée Français and other schools at the bottom of the list seem to be Type 2 charters,
#pulling only schools labeled as "Orleans Parish" 
nolaleap21 = leap21.loc[leap21['School System Name'] == 'Orleans Parish']

In [36]:
print(nolams21.shape)
print(nolaleap21.shape)
#Again, there seem to be 3 schools that either closed or did not report LEAP scores

(77, 41)
(74, 10)


In [37]:
nolams21.head(1)

,School System,School System Name,SIS Submit Site Code,Federal Reporting Site Code,Site Name,Total Enrollment,% Female,% Male,American Indian,Asian,Black,Hispanic,Hawaiian/Pacific Islander,White,Multiple Races (Non-Hispanic),Minority,% Fully English Proficient,% Limited English Proficient,Infants (Sp Ed),Pre-School (Sp Ed),Pre-K (Reg Ed),Kindergarten,Grade 1,Grade 2,Grade 3,Grade 4,Grade 5,Grade 6,Grade 7,Grade 8,Grade 9,Grade T9,Grade 10,Grade 11,Grade 12,Extension Academy,% Economically Disadvantaged,Nonprofit Organization,Charter Type,School System Roll Up Type,Parish Code
1314,R36,Orleans Parish,036011,036011,Mary Bethune Elementary Literature/Technology,709,0.52,0.48,0,0,692,17,0,0,0,709,0.99,0.01,0,6,32,76,75,78,78,78,76,78,79,53,0,0,0,0,0,0,0.85,"Significant Educators, Inc.",Type 3,R36,36


In [38]:
nolaleap21.head(1)

,School System Code,School System Name,Site Code,Site Name,2019 % Mastery+ Grades 3-8,2021 % Mastery+ Grades 3-8,2019-2021 % Mastery+ Change Grades 3-8,2019 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History & Biology,2021 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History & Biology,2019-2021 % Mastery+ Change Grades 3-8 & English I-II & Algebra I & Geometry & US History & Biology
670,R36,Orleans Parish,036011,Mary Bethune Elementary Literature/Technology,30,17,-13,30,17,-13


In [39]:
#Joining SY21 dataframes 
#Using an outer join to preserve those 3 schools that did not report LEAP data
nolasy21 = nolams21.merge(nolaleap21, left_on=['SIS Submit Site Code'], 
                          right_on=['Site Code'], how='outer', indicator=True)

In [40]:
nolasy21.shape
#Join seems to have worked as 3 extra rows are present

(77, 52)

In [41]:
nolasy21.loc[nolasy21['_merge'] == 'left_only']

,School System,School System Name_x,SIS Submit Site Code,Federal Reporting Site Code,Site Name_x,Total Enrollment,% Female,% Male,American Indian,Asian,Black,Hispanic,Hawaiian/Pacific Islander,White,Multiple Races (Non-Hispanic),Minority,% Fully English Proficient,% Limited English Proficient,Infants (Sp Ed),Pre-School (Sp Ed),Pre-K (Reg Ed),Kindergarten,Grade 1,Grade 2,Grade 3,Grade 4,Grade 5,Grade 6,Grade 7,Grade 8,Grade 9,Grade T9,Grade 10,Grade 11,Grade 12,Extension Academy,% Economically Disadvantaged,Nonprofit Organization,Charter Type,School System Roll Up Type,Parish Code,School System Code,School System Name_y,Site Code,Site Name_y,2019 % Mastery+ Grades 3-8,2021 % Mastery+ Grades 3-8,2019-2021 % Mastery+ Change Grades 3-8,2019 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History & Biology,2021 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History & Biology,2019-2021 % Mastery+ Change Grades 3-8 & English I-II & Algebra I & Geometry & US History & Biology,_merge
8,R36,Orleans Parish,036700,036700,Orleans Central Office,330,0.39,0.61,0,7,195,25,0,94,9,236,1.00,0.00,0,330,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.63,NaN,NaN,R36,36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
46,R36,Orleans Parish,3C2002,WZD001,Edward Hynes Charter School - UNO,207,0.47,0.53,0,7,140,2,0,46,12,161,0.99,0.01,0,0,0,104,103,0,0,0,0,0,0,0,0,0,0,0,0,0,0.62,Hynes Charter School Corporation,Type 1,R36,36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
73,R36,Orleans Parish,WC2001,WC2001,Opportunities Academy,65,0.28,0.72,0,2,59,0,0,3,1,62,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,65,0,0.92,Collegiate Academies,Type 1,R36,36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [42]:
#Orleans Central Office, Hynes Charter @ UNO, and Opportunities Academy did not report LEAP data 
nolasy21.loc[nolasy21['_merge'] == 'right_only']

,School System,School System Name_x,SIS Submit Site Code,Federal Reporting Site Code,Site Name_x,Total Enrollment,% Female,% Male,American Indian,Asian,Black,Hispanic,Hawaiian/Pacific Islander,White,Multiple Races (Non-Hispanic),Minority,% Fully English Proficient,% Limited English Proficient,Infants (Sp Ed),Pre-School (Sp Ed),Pre-K (Reg Ed),Kindergarten,Grade 1,Grade 2,Grade 3,Grade 4,Grade 5,Grade 6,Grade 7,Grade 8,Grade 9,Grade T9,Grade 10,Grade 11,Grade 12,Extension Academy,% Economically Disadvantaged,Nonprofit Organization,Charter Type,School System Roll Up Type,Parish Code,School System Code,School System Name_y,Site Code,Site Name_y,2019 % Mastery+ Grades 3-8,2021 % Mastery+ Grades 3-8,2019-2021 % Mastery+ Change Grades 3-8,2019 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History & Biology,2021 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History & Biology,2019-2021 % Mastery+ Change Grades 3-8 & English I-II & Algebra I & Geometry & US History & Biology,_merge


In [43]:
#Again, no schools present in the LEAP dataset that are not already present in the multistat
print(nolasy21['SIS Submit Site Code'].isna().sum()) #How many nulls are from the multistat file? 
print(nolasy21['Site Code'].isna().sum()) #How many nulls are from the LEAP file? 

0
3


In [44]:
#Since there are no nulls present in the multistat file, we can be confident in dropping the "Site Code" column
#Cleaning up SY21 columns
#Dropping redundant columns - FedSiteCd, School System, School System Name_x and School System Name_y 
#(they are all Orleans)
#Alsop dropping _merge, Parish Code, and School System Code, Site Code, and Site Name as per the check above 
nolasy21 = nolasy21.drop(columns=['School System','School System Name_x', 'Federal Reporting Site Code',
                                  'School System Name_y',
                                '_merge','Parish Code','School System Code','Site Code','Site Name_y'], axis=1)

In [45]:
nolasy21.shape

(77, 43)

In [46]:
#Add 'SY21' to all column names 
nolasy21 = nolasy21.add_prefix('sy21_')

In [47]:
#Repeat process one last time for SY22
leap22.head()

,School System Code,School System Name,Site Code,Site Name,2021 % Mastery+ Grades 3-8,2022 % Mastery+ Grades 3-8,2021-2022 % Mastery+ Change Grades 3-8,2021 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History & Biology,2022 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History & Biology,2021-2022 % Mastery+ Change Grades 3-8 & English I-II & Algebra I & Geometry & US History & Biology
0,001,Acadia Parish,001001,Armstrong Middle School,23,16,-7,23,16,-7
1,001,Acadia Parish,001002,Branch Elementary School,44,44,0,44,44,0
2,001,Acadia Parish,001004,Church Point Elementary School,31,28,-3,31,28,-3
3,001,Acadia Parish,001005,Church Point High School,NR,NR,NR,23,20,-3
4,001,Acadia Parish,001006,Church Point Middle School,22,26,4,22,26,4


In [48]:
leap22['School System Name'].value_counts()

Jefferson Parish                                      81
East Baton Rouge Parish                               80
Orleans Parish                                        74
Caddo Parish                                          57
Calcasieu Parish                                      52
St. Tammany Parish                                    50
Lafayette Parish                                      44
Livingston Parish                                     42
Rapides Parish                                        41
Ouachita Parish                                       33
Bossier Parish                                        32
Tangipahoa Parish                                     31
St. Landry Parish                                     31
Ascension Parish                                      30
Terrebonne Parish                                     29
Lafourche Parish                                      26
Acadia Parish                                         25
New Orleans Archdiocese        

In [49]:
#Again, not including Type 2 charters in final dataframe
nolaleap22 = leap22.loc[leap22['School System Name'] == 'Orleans Parish']

In [50]:
print(nolams22.shape)
print(nolaleap22.shape)
#Again, there seem to be 3 schools that either closed or did not report LEAP scores

(77, 41)
(74, 10)


In [51]:
nolams22.head(1)

,School System,School System Name,SIS Submit Site Code,Federal Reporting Site Code,Site Name,Total Enrollment,% Female,% Male,American Indian,Asian,Black,Hispanic,Hawaiian/Pacific Islander,White,Multiple Races (Non-Hispanic),Minority,% Fully English Proficient,% Limited English Proficient,Infants (Sp Ed),Pre-School (Sp Ed),Pre-K (Reg Ed),Kindergarten,Grade 1,Grade 2,Grade 3,Grade 4,Grade 5,Grade 6,Grade 7,Grade 8,Grade 9,Grade T9,Grade 10,Grade 11,Grade 12,Extension Academy,% Economically Disadvantaged,Nonprofit Organization,Charter Type,School System Roll Up Type,Parish Code
740,R36,Orleans Parish,036011,036011,Mary Bethune Elementary Literature/Technology,699,0.51,0.49,0,0,678,20,1,0,0,699,0.98,0.02,0,8,32,70,74,72,66,70,77,74,78,78,0,0,0,0,0,0,0.93,"Significant Educators, Inc.",Type 3,R36,36


In [52]:
nolaleap22.head(1)

,School System Code,School System Name,Site Code,Site Name,2021 % Mastery+ Grades 3-8,2022 % Mastery+ Grades 3-8,2021-2022 % Mastery+ Change Grades 3-8,2021 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History & Biology,2022 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History & Biology,2021-2022 % Mastery+ Change Grades 3-8 & English I-II & Algebra I & Geometry & US History & Biology
674,R36,Orleans Parish,036011,Mary Bethune Elementary Literature/Technology,17,20,3,17,21,4


In [53]:
#Joining SY22 dataframes 
#Using an outer join to preserve those 3 schools that did not report LEAP data
nolasy22 = nolams22.merge(nolaleap22, left_on=['SIS Submit Site Code'], 
                          right_on=['Site Code'], how='outer', indicator=True)

In [54]:
nolasy22.shape

(77, 52)

In [55]:
nolasy22.loc[nolasy22['_merge'] == 'left_only']
#Again, Central Office, Hynes @ UNO, and Opportunities Academy did not report LEAP data 

,School System,School System Name_x,SIS Submit Site Code,Federal Reporting Site Code,Site Name_x,Total Enrollment,% Female,% Male,American Indian,Asian,Black,Hispanic,Hawaiian/Pacific Islander,White,Multiple Races (Non-Hispanic),Minority,% Fully English Proficient,% Limited English Proficient,Infants (Sp Ed),Pre-School (Sp Ed),Pre-K (Reg Ed),Kindergarten,Grade 1,Grade 2,Grade 3,Grade 4,Grade 5,Grade 6,Grade 7,Grade 8,Grade 9,Grade T9,Grade 10,Grade 11,Grade 12,Extension Academy,% Economically Disadvantaged,Nonprofit Organization,Charter Type,School System Roll Up Type,Parish Code,School System Code,School System Name_y,Site Code,Site Name_y,2021 % Mastery+ Grades 3-8,2022 % Mastery+ Grades 3-8,2021-2022 % Mastery+ Change Grades 3-8,2021 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History & Biology,2022 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History & Biology,2021-2022 % Mastery+ Change Grades 3-8 & English I-II & Algebra I & Geometry & US History & Biology,_merge
5,R36,Orleans Parish,036700,036700,Orleans Central Office,365,0.35,0.65,0,5,235,20,0,96,9,269,1,0,0,365,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.62,NaN,NaN,R36,36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
42,R36,Orleans Parish,3C2002,WZD001,Edward Hynes Charter School - UNO,290,0.43,0.57,1,9,190,3,0,66,21,224,0.98,0.02,0,0,0,99,100,91,0,0,0,0,0,0,0,0,0,0,0,0,0.59,Hynes Charter School Corporation,Type 1,R36,36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
73,R36,Orleans Parish,WC2001,WC2001,Opportunities Academy,70,0.31,0.69,0,2,64,1,0,2,1,68,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,70,0,0.96,Collegiate Academies,Type 1,R36,36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [56]:
nolasy22.loc[nolasy22['_merge'] == 'right_only']

,School System,School System Name_x,SIS Submit Site Code,Federal Reporting Site Code,Site Name_x,Total Enrollment,% Female,% Male,American Indian,Asian,Black,Hispanic,Hawaiian/Pacific Islander,White,Multiple Races (Non-Hispanic),Minority,% Fully English Proficient,% Limited English Proficient,Infants (Sp Ed),Pre-School (Sp Ed),Pre-K (Reg Ed),Kindergarten,Grade 1,Grade 2,Grade 3,Grade 4,Grade 5,Grade 6,Grade 7,Grade 8,Grade 9,Grade T9,Grade 10,Grade 11,Grade 12,Extension Academy,% Economically Disadvantaged,Nonprofit Organization,Charter Type,School System Roll Up Type,Parish Code,School System Code,School System Name_y,Site Code,Site Name_y,2021 % Mastery+ Grades 3-8,2022 % Mastery+ Grades 3-8,2021-2022 % Mastery+ Change Grades 3-8,2021 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History & Biology,2022 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History & Biology,2021-2022 % Mastery+ Change Grades 3-8 & English I-II & Algebra I & Geometry & US History & Biology,_merge


In [57]:
#Again, no schools present in the LEAP dataset that are not already present in the multistat
print(nolasy22['SIS Submit Site Code'].isna().sum()) #How many nulls are from the multistat file? 
print(nolasy22['Site Code'].isna().sum()) #How many nulls are from the LEAP file? 

0
3


In [58]:
#Since there are no nulls present in the multistat file, we can be confident in dropping the "Site Code" column
#Cleaning up SY22 columns
#Dropping redundant columns - FedSiteCd, School System, School System Name_x and School System Name_y 
#(they are all Orleans)
#Alsop dropping _merge, Parish Code, and School System Code, Site Code, and Site Name as per the check above 
nolasy22 = nolasy22.drop(columns=['School System','School System Name_x', 'Federal Reporting Site Code',
                                  'School System Name_y',
                                '_merge','Parish Code','School System Code','Site Code','Site Name_y'], axis=1)

In [59]:
nolasy22.shape

(77, 43)

In [60]:
#Lastly, add 'SY22' to all column names 
nolasy22 = nolasy22.add_prefix('sy22_')

In [61]:
print(nolasy19.duplicated(subset=['sy19_SiteCd']).sum())
print(nolasy19.duplicated(subset=['sy19_SiteName']).sum())

0
0


In [62]:
print(nolasy21.duplicated(subset=['sy21_SIS Submit Site Code']).sum())
print(nolasy21.duplicated(subset=['sy21_Site Name_x']).sum())

0
0


### II. Answering Questions 2-4
2. Please list the new site codes and associated school names that enrolled student in 2019, but no longer enroll students in 2021.
3. Please list the new site codes in Orleans Parish and associated school names that enrolled students in 2021 that did not exist in 2019.

In [63]:
#Join SY19 and SY21 data
sy19to21 = nolasy19.merge(nolasy21, left_on=['sy19_SiteCd'], 
                          right_on=['sy21_SIS Submit Site Code'], how='outer', indicator=True)

In [64]:
sy19to21.shape

(93, 86)

In [65]:
potential_sy21_closures = sy19to21.loc[sy19to21['_merge'] == 'left_only']
#School codes that appear only in the FY19 datafiles

In [66]:
potential_sy21_closures.shape
#16 schools appear only in the FY19 datafiles but not in the FY21 datafiles

(16, 86)

In [67]:
potential_sy21_new_schools = sy19to21.loc[sy19to21['_merge'] == 'right_only']
#School codes that appear only in the FY21 datafiles 

In [68]:
potential_sy21_new_schools.shape
#14 schools appear in the FY21 datafile but not in the FY19 datafile

(14, 86)

In [69]:
potential_sy21_closures = potential_sy21_closures[['sy19_SiteCd','sy19_SiteName']]

In [70]:
potential_sy21_new_schools = potential_sy21_new_schools[['sy21_SIS Submit Site Code','sy21_Site Name_x']]

In [71]:
potential_openers_list = potential_sy21_new_schools['sy21_Site Name_x'].tolist()

In [72]:
#Use fuzzy matching since school names seem to differ slightly 

def fuzzy(col, potential_openers_list, score):
    fuzzy_words, score = process.extractOne(col, potential_openers_list, scorer=score)
    if score<75:
        return 'no_high_matches'
    else:
        return fuzzy_words,score
    
potential_sy21_closures['fuzzy_match'] = potential_sy21_closures['sy19_SiteName'].apply(fuzzy, potential_openers_list=potential_openers_list, score=fuzz.ratio)



In [73]:
potential_sy21_closures

,sy19_SiteCd,sy19_SiteName,fuzzy_match
1,036035,Warren Easton Senior High School,"(Warren Easton Charter High School, 86)"
2,036060,Edward Hynes Charter School,"(Edward Hynes Charter School - UNO, 90)"
7,036192,Foundation Preparatory,"(Foundation Preparatory Academy, 85)"
8,036193,Cypress Academy,no_high_matches
12,300001,Pierre A. Capdau Charter School at Avery Alexa...,"(Pierre A. Capdau Charter School, 77)"
13,300002,Nelson Elementary School,no_high_matches
14,300003,John F. Kennedy High School,"(John F. Kennedy High School, 100)"
20,367001,Edgar P. Harney Spirit of Excellence Academy,no_high_matches
24,369005,ReNEW Accelerated High School,"(New Orleans Accelerated High School, 84)"
36,393001,Lafayette Academy,no_high_matches


In [74]:
sy19_closures = potential_sy21_closures.loc[potential_sy21_closures['fuzzy_match'] == 'no_high_matches']
sy19_closure_list = dict(zip(sy19_closures.sy19_SiteCd, sy19_closures.sy19_SiteName))

In [75]:
sy19_closure_list

{'036193': 'Cypress Academy',
 '300002': 'Nelson Elementary School',
 '367001': 'Edgar P. Harney Spirit of Excellence Academy',
 '393001': 'Lafayette Academy',
 '395003': 'William J. Fischer Accelerated Academy',
 '395004': 'McDonogh #32 Literacy Charter School',
 '399003': 'Joseph S. Clark Preparatory High School',
 'W32001': 'Joseph A. Craig Charter School'}

In [76]:
len(sy19_closure_list)

8

Looks like 8 schools no longer enroll students in 2021. 

But fuzzy matching isn't perfect - the function didn't pick up on ReNEW Accelerated closing and turning into New Orleans Accelerated High. It also should not include Lafayette Academy, who simply had to move out of their building (if I remember the news correctly). 

In [77]:
sy19_closure_list.update({'369005':'ReNEW Accelerated High School'})
del sy19_closure_list['393001']

In [78]:
#Codes and names of schools that enrolled students in 2019 but not in 2021
sy19_closure_list

{'036193': 'Cypress Academy',
 '300002': 'Nelson Elementary School',
 '367001': 'Edgar P. Harney Spirit of Excellence Academy',
 '395003': 'William J. Fischer Accelerated Academy',
 '395004': 'McDonogh #32 Literacy Charter School',
 '399003': 'Joseph S. Clark Preparatory High School',
 'W32001': 'Joseph A. Craig Charter School',
 '369005': 'ReNEW Accelerated High School'}

In [79]:
code_changers = potential_sy21_closures.loc[potential_sy21_closures['fuzzy_match'] != 'no_high_matches']
code_change_list = dict(zip(code_changers.sy19_SiteCd, code_changers.sy19_SiteName))

In [80]:
code_change_list

{'036035': 'Warren Easton Senior High School',
 '036060': 'Edward Hynes Charter School',
 '036192': 'Foundation Preparatory',
 '300001': 'Pierre A. Capdau Charter School at Avery Alexander',
 '300003': 'John F. Kennedy High School',
 '369005': 'ReNEW Accelerated High School',
 '393002': 'Esperanza Charter School',
 '3A5001': 'Mary D. Coghill Charter School'}

I know ReNEW Accelerated High School in actuality closed. I also know Lafayette Academy should be on this list, which the function also missed. 

In [81]:
code_change_list.update({'393001':'Lafayette Academy'})
del code_change_list['369005']

In [82]:
#Codes and names of schools that switched site codes from 2019 to 2021 
code_change_list

{'036035': 'Warren Easton Senior High School',
 '036060': 'Edward Hynes Charter School',
 '036192': 'Foundation Preparatory',
 '300001': 'Pierre A. Capdau Charter School at Avery Alexander',
 '300003': 'John F. Kennedy High School',
 '393002': 'Esperanza Charter School',
 '3A5001': 'Mary D. Coghill Charter School',
 '393001': 'Lafayette Academy'}

Above are the names and site codes of the 8 schools that changed site codes between 2019 and 2021.

In [83]:
len(code_change_list)

8

In [84]:
changers_list = list(code_change_list.values())

In [85]:
changers_list

['Warren Easton Senior High School',
 'Edward Hynes Charter School',
 'Foundation Preparatory',
 'Pierre A. Capdau Charter School at Avery Alexander',
 'John F. Kennedy High School',
 'Esperanza Charter School',
 'Mary D. Coghill Charter School',
 'Lafayette Academy']

In [86]:
#Repeat the process to find the new schools from 2021 which were not open in 2019 
def fuzzy2(col, changers_list, score):
    fuzzy_words, score = process.extractOne(col, changers_list, scorer=score)
    if score<75:
        return 'no_high_matches'
    else:
        return fuzzy_words,score
    
potential_sy21_new_schools['fuzzy_match'] = potential_sy21_new_schools['sy21_Site Name_x'].apply(fuzzy2, changers_list=changers_list, score=fuzz.ratio)



In [87]:
#If there is NOT a match, it means that school started enrolling students in 2021 
#Except for Lafayette Academy, which the function keeps missing
potential_sy21_new_schools

,sy21_SIS Submit Site Code,sy21_Site Name_x,fuzzy_match
79,036021,Mary D. Coghill Elementary School,"(Mary D. Coghill Charter School, 79)"
80,360003,New Orleans Accelerated High School,no_high_matches
81,398009,John F. Kennedy High School,"(John F. Kennedy High School, 100)"
82,3C2001,Edward Hynes Charter School - Lakeview,"(Edward Hynes Charter School, 83)"
83,3C2002,Edward Hynes Charter School - UNO,"(Edward Hynes Charter School, 90)"
84,3C3001,Foundation Preparatory Academy,"(Foundation Preparatory, 85)"
85,3C3002,Lafayette Academy Charter School,no_high_matches
86,3C3003,Esperanza Charter School,"(Esperanza Charter School, 100)"
87,WBW001,Living School,no_high_matches
88,WBZ001,McDonogh 35 Senior High School,no_high_matches


In [88]:
sy21_new_schools = potential_sy21_new_schools.loc[potential_sy21_new_schools['fuzzy_match'] == 'no_high_matches']

In [89]:
sy21_new_schools_list = sy21_new_schools.set_index('sy21_SIS Submit Site Code').to_dict()['sy21_Site Name_x']

In [90]:
sy21_new_schools_list

{'360003': 'New Orleans Accelerated High School',
 '3C3002': 'Lafayette Academy Charter School',
 'WBW001': 'Living School',
 'WBZ001': 'McDonogh 35 Senior High School',
 'WC2001': 'Opportunities Academy',
 'WC3001': 'IDEA Oscar Dunn'}

In [91]:
#Drop Lafayette from list, as we know that in actuality they just changed buildings 
#Add Hynes Charter - UNO to list as this is a new campus 
#Fuzzy matching is not picking up on this
sy21_new_schools_list.update({'3C2002':'Edward Hynes Charter School - UNO'})
del sy21_new_schools_list['3C3002']

In [92]:
#Codes and names of schools that served students in 2021 but not 2019
sy21_new_schools_list

{'360003': 'New Orleans Accelerated High School',
 'WBW001': 'Living School',
 'WBZ001': 'McDonogh 35 Senior High School',
 'WC2001': 'Opportunities Academy',
 'WC3001': 'IDEA Oscar Dunn',
 '3C2002': 'Edward Hynes Charter School - UNO'}

Above are the 6 schools that did not enroll students in 2019, but did so in 2021. After I answered Questions 2,3, and 4, I went back to cleaning the dataframe.

### III. Longitudinal Merging

In [93]:
#Schools that closed in SY19 have NANs in SY21 columns 
#Schools that opened in SY21 have NANs in SY19 columns
#Schools that changed codes have data in all columns
#Presumably the schools that have changed codes have duplicate rows. We need to 
#get the data for these schools on one row so each school has just one row 
#Making list of all potential names to find all rows associated with schools that have changed codes 
fuzzymatches = sorted(potential_sy21_closures['sy19_SiteName'].tolist() + potential_sy21_new_schools['sy21_Site Name_x'].tolist())

In [94]:
fuzzymatches

['Cypress Academy',
 'Edgar P. Harney Spirit of Excellence Academy',
 'Edward Hynes Charter School',
 'Edward Hynes Charter School - Lakeview',
 'Edward Hynes Charter School - UNO',
 'Esperanza Charter School',
 'Esperanza Charter School',
 'Foundation Preparatory',
 'Foundation Preparatory Academy',
 'IDEA Oscar Dunn',
 'John F. Kennedy High School',
 'John F. Kennedy High School',
 'Joseph A. Craig Charter School',
 'Joseph S. Clark Preparatory High School',
 'Lafayette Academy',
 'Lafayette Academy Charter School',
 'Living School',
 'Mary D. Coghill Charter School',
 'Mary D. Coghill Elementary School',
 'McDonogh #32 Literacy Charter School',
 'McDonogh 35 Senior High School',
 'Nelson Elementary School',
 'New Orleans Accelerated High School',
 'Opportunities Academy',
 'Pierre A. Capdau Charter School',
 'Pierre A. Capdau Charter School at Avery Alexander',
 'ReNEW Accelerated High School',
 'Warren Easton Charter High School',
 'Warren Easton Senior High School',
 'William J. F

In [95]:
sy19to21.head(1)

,sy19_SiteCd,sy19_SiteName,sy19_Total Students,sy19_%Female,sy19_%Male,sy19_AmInd,sy19_Asian,sy19_Black,sy19_Hispanic,sy19_HawPI,sy19_White,sy19_Multiple,sy19_Minority,sy19_%Fully-EP,sy19_%LEP,sy19_Infants SpEd,sy19_PreSchool SpEd,sy19_PreK,sy19_Kindergarten,sy19_Grade1,sy19_Grade2,sy19_Grade3,sy19_Grade4,sy19_Grade5,sy19_Grade6,sy19_Grade7,sy19_Grade8,sy19_Grade9,sy19_GradeT9,sy19_Grade10,sy19_Grade11,sy19_Grade12,sy19_ED%,sy19_Nonprofit,sy19_Charter Type,sy19_RollUpType,sy19_2018 % Mastery+ Grades 3-8,sy19_2019 % Mastery+ Grades 3-8,sy19_2018-2019 % Mastery+ Change Grades 3-8,sy19_2018 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History,sy19_2019 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History,sy19_2018-2019 % Mastery+ Change Grades 3-8 & English I-II & Algebra I & Geometry & US History,sy21_SIS Submit Site Code,sy21_Site Name_x,sy21_Total Enrollment,sy21_% Female,sy21_% Male,sy21_American Indian,sy21_Asian,sy21_Black,sy21_Hispanic,sy21_Hawaiian/Pacific Islander,sy21_White,sy21_Multiple Races (Non-Hispanic),sy21_Minority,sy21_% Fully English Proficient,sy21_% Limited English Proficient,sy21_Infants (Sp Ed),sy21_Pre-School (Sp Ed),sy21_Pre-K (Reg Ed),sy21_Kindergarten,sy21_Grade 1,sy21_Grade 2,sy21_Grade 3,sy21_Grade 4,sy21_Grade 5,sy21_Grade 6,sy21_Grade 7,sy21_Grade 8,sy21_Grade 9,sy21_Grade T9,sy21_Grade 10,sy21_Grade 11,sy21_Grade 12,sy21_Extension Academy,sy21_% Economically Disadvantaged,sy21_Nonprofit Organization,sy21_Charter Type,sy21_School System Roll Up Type,sy21_2019 % Mastery+ Grades 3-8,sy21_2021 % Mastery+ Grades 3-8,sy21_2019-2021 % Mastery+ Change Grades 3-8,sy21_2019 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History & Biology,sy21_2021 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History & Biology,sy21_2019-2021 % Mastery+ Change Grades 3-8 & English I-II & Algebra I & Geometry & US History & Biology,_merge
0,036011,Mary Bethune Elementary Literature/Technology,688.00,0.52,0.48,0.00,2.00,669.00,15.00,0.00,2.00,0.00,686.00,0.99,0.01,0.00,2.00,38.00,78.00,78.00,80.00,81.00,79.00,82.00,54.00,55.00,61.00,0.00,0.00,0.00,0.00,0.00,0.82,NaN,NaN,R36,31,33,2,31,34,3,036011,Mary Bethune Elementary Literature/Technology,709,0.52,0.48,0,0,692,17,0,0,0,709,0.99,0.01,0,6,32,76,75,78,78,78,76,78,79,53,0,0,0,0,0,0,0.85,"Significant Educators, Inc.",Type 3,R36,30,17,-13,30,17,-13,both


In [96]:
code_change_list

{'036035': 'Warren Easton Senior High School',
 '036060': 'Edward Hynes Charter School',
 '036192': 'Foundation Preparatory',
 '300001': 'Pierre A. Capdau Charter School at Avery Alexander',
 '300003': 'John F. Kennedy High School',
 '393002': 'Esperanza Charter School',
 '3A5001': 'Mary D. Coghill Charter School',
 '393001': 'Lafayette Academy'}

In [97]:
keywords = ['Easton','Hynes','Capdau','Foundation','Kennedy','Esperanza','Coghill', 'Lafayette']

In [98]:
#Pulling all rows with schools that switched school codes 
switchers = sy19to21[sy19to21.stack().str.contains('|'.join(keywords)).any(level=0)]

In [99]:
#Dropping Hynes UNO from school code change group, as this is a brand new campus 
switchers_clean = switchers.drop(labels=[13,83], axis=0) #Nelson is at position 13; Hynes UNO is at position 83

In [100]:
switchers_clean.shape

(16, 86)

In [101]:
switchers_clean = switchers_clean.sort_values(by='sy21_Site Name_x')

In [102]:
sy19switchers = switchers_clean.filter(regex='^sy19', axis=1).dropna(how='all').sort_values(by='sy19_SiteName')

In [103]:
sy19switchers.shape

(8, 42)

In [104]:
sy21switchers = switchers_clean.filter(regex='^sy21', axis=1).dropna(how='all').sort_values(by='sy21_Site Name_x')

In [105]:
sy21switchers.shape

(8, 43)

In [106]:
sy19_school_name_list = sy19switchers['sy19_SiteName'].tolist()

In [107]:
sy19_school_name_list

['Edward Hynes Charter School',
 'Esperanza Charter School',
 'Foundation Preparatory',
 'John F. Kennedy High School',
 'Lafayette Academy',
 'Mary D. Coghill Charter School',
 'Pierre A. Capdau Charter School at Avery Alexander',
 'Warren Easton Senior High School']

In [108]:
sy21_school_name_list = sy21switchers['sy21_Site Name_x'].tolist()

In [109]:
sy21_school_name_list

['Edward Hynes Charter School - Lakeview',
 'Esperanza Charter School',
 'Foundation Preparatory Academy',
 'John F. Kennedy High School',
 'Lafayette Academy Charter School',
 'Mary D. Coghill Elementary School',
 'Pierre A. Capdau Charter School',
 'Warren Easton Charter High School']

In [110]:
sy19switchers = sy19switchers.replace('Edward Hynes Charter School', 'Edward Hynes Charter School - Lakeview').replace('Foundation Preparatory','Foundation Preparatory Academy').replace('Lafayette Academy','Lafayette Academy Charter School').replace('Mary D. Coghill Charter School','Mary D. Coghill Elementary School').replace('Warren Easton Senior High School','Warren Easton Charter High School').replace('Pierre A. Capdau Charter School at Avery Alexander','Pierre A. Capdau Charter School')


In [111]:
switchers_19to21final = sy19switchers.merge(sy21switchers, how='inner', left_on='sy19_SiteName', right_on ='sy21_Site Name_x')

In [112]:
switchers_19to21final.shape

(8, 85)

In [113]:
switchers_19to21final.head(1)

,sy19_SiteCd,sy19_SiteName,sy19_Total Students,sy19_%Female,sy19_%Male,sy19_AmInd,sy19_Asian,sy19_Black,sy19_Hispanic,sy19_HawPI,sy19_White,sy19_Multiple,sy19_Minority,sy19_%Fully-EP,sy19_%LEP,sy19_Infants SpEd,sy19_PreSchool SpEd,sy19_PreK,sy19_Kindergarten,sy19_Grade1,sy19_Grade2,sy19_Grade3,sy19_Grade4,sy19_Grade5,sy19_Grade6,sy19_Grade7,sy19_Grade8,sy19_Grade9,sy19_GradeT9,sy19_Grade10,sy19_Grade11,sy19_Grade12,sy19_ED%,sy19_Nonprofit,sy19_Charter Type,sy19_RollUpType,sy19_2018 % Mastery+ Grades 3-8,sy19_2019 % Mastery+ Grades 3-8,sy19_2018-2019 % Mastery+ Change Grades 3-8,sy19_2018 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History,sy19_2019 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History,sy19_2018-2019 % Mastery+ Change Grades 3-8 & English I-II & Algebra I & Geometry & US History,sy21_SIS Submit Site Code,sy21_Site Name_x,sy21_Total Enrollment,sy21_% Female,sy21_% Male,sy21_American Indian,sy21_Asian,sy21_Black,sy21_Hispanic,sy21_Hawaiian/Pacific Islander,sy21_White,sy21_Multiple Races (Non-Hispanic),sy21_Minority,sy21_% Fully English Proficient,sy21_% Limited English Proficient,sy21_Infants (Sp Ed),sy21_Pre-School (Sp Ed),sy21_Pre-K (Reg Ed),sy21_Kindergarten,sy21_Grade 1,sy21_Grade 2,sy21_Grade 3,sy21_Grade 4,sy21_Grade 5,sy21_Grade 6,sy21_Grade 7,sy21_Grade 8,sy21_Grade 9,sy21_Grade T9,sy21_Grade 10,sy21_Grade 11,sy21_Grade 12,sy21_Extension Academy,sy21_% Economically Disadvantaged,sy21_Nonprofit Organization,sy21_Charter Type,sy21_School System Roll Up Type,sy21_2019 % Mastery+ Grades 3-8,sy21_2021 % Mastery+ Grades 3-8,sy21_2019-2021 % Mastery+ Change Grades 3-8,sy21_2019 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History & Biology,sy21_2021 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History & Biology,sy21_2019-2021 % Mastery+ Change Grades 3-8 & English I-II & Algebra I & Geometry & US History & Biology
0,036060,Edward Hynes Charter School - Lakeview,717.00,0.53,0.47,0.00,32.00,252.00,30.00,0.00,361.00,42.00,356.00,0.97,0.03,0.00,17.00,0.00,83.00,84.00,79.00,79.00,72.00,80.00,77.00,76.00,70.00,0.00,0.00,0.00,0.00,0.00,0.35,Hynes Charter School Corporation,Type 3,R36,61,63,2,60,63,3,3C2001,Edward Hynes Charter School - Lakeview,712,0.53,0.47,0,35,236,30,0,369,42,343,0.97,0.03,0,16,0,78,80,84,79,77,78,76,77,67,0,0,0,0,0,0,0.42,Hynes Charter School Corporation,Type 1,R36,NR,56,NR,NR,55,NR


In [114]:
sy19to21.shape

(93, 86)

In [115]:
#Replacing rows in 19 to 21 table with correct rows for schools that have switched codes 
#First need to drop the duplicate rows for both years
sy19to21_updated = sy19to21[~sy19to21['sy19_SiteCd'].isin(switchers_19to21final['sy19_SiteCd'])]

In [116]:
sy19to21_updated.shape

(85, 86)

In [117]:
sy19to21_updated2 = sy19to21_updated[~sy19to21_updated['sy21_SIS Submit Site Code'].isin(switchers_19to21final['sy21_SIS Submit Site Code'])]

In [118]:
sy19to21_updated2.shape

(77, 86)

In [119]:
#Now need to add the correct rows back in 
sy19to21 = sy19to21_updated2.append(switchers_19to21final)

In [120]:
sy19to21.shape
#Confirmed that rows for the schools that have switched codes are added back in 

(85, 86)

In [121]:
#Need to drop indicator column in order for the below merge to work 
sy19to21 = sy19to21.drop(columns=['_merge'])

In [122]:
nolasy22.shape

(77, 43)

In [123]:
#Now, we can join the 19/21 dataset with the 22 dataset 
sy19to22 = sy19to21.merge(nolasy22, left_on=['sy21_SIS Submit Site Code'], 
                          right_on=['sy22_SIS Submit Site Code'], how='outer', indicator=True)

In [124]:
sy19to22.shape

(90, 129)

In [125]:
potential_sy22_closures = sy19to22.loc[sy19to22['_merge'] == 'left_only']
#School codes that appear only in the FY19/FY21 datafiles

In [126]:
potential_sy22_closures.shape
#13 site codes appear only in the FY19/21 datafiles but not in the FY21 datafiles

(13, 129)

In [127]:
potential_sy22_new_schools = sy19to22.loc[sy19to22['_merge'] == 'right_only']
#School codes that appear only in the FY22 datafiles 

In [128]:
potential_sy22_new_schools.shape
#5 schools appear in the FY21 datafile but not in the FY19 datafile

(5, 129)

In [129]:
potential_sy22_closures = potential_sy22_closures[['sy21_SIS Submit Site Code','sy21_Site Name_x']]
potential_sy22_new_schools = potential_sy22_new_schools[['sy22_SIS Submit Site Code','sy22_Site Name_x']]

In [130]:
sy22_potential_openers_list = potential_sy22_new_schools['sy22_Site Name_x'].tolist()

In [131]:
#Schools that may have closed by 2022
potential_sy22_closures

,sy21_SIS Submit Site Code,sy21_Site Name_x
1,036088,McDonogh #35 College Preparatory School
4,036189,Homer A. Plessy Community School
5,NaN,NaN
6,NaN,NaN
7,NaN,NaN
8,NaN,NaN
9,NaN,NaN
10,NaN,NaN
11,NaN,NaN
12,NaN,NaN


In [132]:
#Schools that may have started serving students in 2022
potential_sy22_new_schools

,sy22_SIS Submit Site Code,sy22_Site Name_x
85,036200,The Delores Taylor Arthur School for Young Men
86,3C2003,Hynes Parkview
87,3C6001,Homer Plessy Community School
88,3C7001,Rooted School
89,3C8001,YACS at Lawrence D. Crocker


Between 2021 and 2022: 

Closures: 
- Lawrence D. Crocker College Prep (385003)
- Mary D. Coghill (036021)

School Code Changes: 
- 2021 - Homer A. Plessy Community School (036189)
- 2022 - Homer Plessy Community School (3C6001)
- 2021 - Rooted School (036198) 
- 2022 - Rooted School (3C7001) 

Openers: 
- The Deleres Taylor Arthur School for Young Men (036200)
- Hynes Parkview (3C2003)
- YACS at Lawrence D. Crocker (3C8001)

Unsure: McDonogh #35 has two rows and two distinct school codes for 2021. Each row contains different data, so I'm unsure what to do here: 
- 2021 - McDonogh #35 College Preparatory School (036088)
- 2021 and 2022 - McDonogh 35 Senior High School (WBZ001) 

In [133]:
#There are only two other schools that seemed to switch codes in 2022, so I can
#handle these duplicates manually without using fuzzy 
keywords = ['Plessy','Rooted']

In [134]:
#Pulling all rows with schools that switched school codes 
sy21to22_switchers = sy19to22[sy19to22.stack().str.contains('|'.join(keywords)).any(level=0)]

In [135]:
sy21to22_switchers.shape

(4, 129)

In [136]:
sy21to22_switchers.head(1)

,sy19_SiteCd,sy19_SiteName,sy19_Total Students,sy19_%Female,sy19_%Male,sy19_AmInd,sy19_Asian,sy19_Black,sy19_Hispanic,sy19_HawPI,sy19_White,sy19_Multiple,sy19_Minority,sy19_%Fully-EP,sy19_%LEP,sy19_Infants SpEd,sy19_PreSchool SpEd,sy19_PreK,sy19_Kindergarten,sy19_Grade1,sy19_Grade2,sy19_Grade3,sy19_Grade4,sy19_Grade5,sy19_Grade6,sy19_Grade7,sy19_Grade8,sy19_Grade9,sy19_GradeT9,sy19_Grade10,sy19_Grade11,sy19_Grade12,sy19_ED%,sy19_Nonprofit,sy19_Charter Type,sy19_RollUpType,sy19_2018 % Mastery+ Grades 3-8,sy19_2019 % Mastery+ Grades 3-8,sy19_2018-2019 % Mastery+ Change Grades 3-8,sy19_2018 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History,sy19_2019 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History,sy19_2018-2019 % Mastery+ Change Grades 3-8 & English I-II & Algebra I & Geometry & US History,sy21_SIS Submit Site Code,sy21_Site Name_x,sy21_Total Enrollment,sy21_% Female,sy21_% Male,sy21_American Indian,sy21_Asian,sy21_Black,sy21_Hispanic,sy21_Hawaiian/Pacific Islander,sy21_White,sy21_Multiple Races (Non-Hispanic),sy21_Minority,sy21_% Fully English Proficient,sy21_% Limited English Proficient,sy21_Infants (Sp Ed),sy21_Pre-School (Sp Ed),sy21_Pre-K (Reg Ed),sy21_Kindergarten,sy21_Grade 1,sy21_Grade 2,sy21_Grade 3,sy21_Grade 4,sy21_Grade 5,sy21_Grade 6,sy21_Grade 7,sy21_Grade 8,sy21_Grade 9,sy21_Grade T9,sy21_Grade 10,sy21_Grade 11,sy21_Grade 12,sy21_Extension Academy,sy21_% Economically Disadvantaged,sy21_Nonprofit Organization,sy21_Charter Type,sy21_School System Roll Up Type,sy21_2019 % Mastery+ Grades 3-8,sy21_2021 % Mastery+ Grades 3-8,sy21_2019-2021 % Mastery+ Change Grades 3-8,sy21_2019 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History & Biology,sy21_2021 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History & Biology,sy21_2019-2021 % Mastery+ Change Grades 3-8 & English I-II & Algebra I & Geometry & US History & Biology,sy22_SIS Submit Site Code,sy22_Site Name_x,sy22_Total Enrollment,sy22_% Female,sy22_% Male,sy22_American Indian,sy22_Asian,sy22_Black,sy22_Hispanic,sy22_Hawaiian/Pacific Islander,sy22_White,sy22_Multiple Races (Non-Hispanic),sy22_Minority,sy22_% Fully English Proficient,sy22_% Limited English Proficient,sy22_Infants (Sp Ed),sy22_Pre-School (Sp Ed),sy22_Pre-K (Reg Ed),sy22_Kindergarten,sy22_Grade 1,sy22_Grade 2,sy22_Grade 3,sy22_Grade 4,sy22_Grade 5,sy22_Grade 6,sy22_Grade 7,sy22_Grade 8,sy22_Grade 9,sy22_Grade T9,sy22_Grade 10,sy22_Grade 11,sy22_Grade 12,sy22_Extension Academy,sy22_% Economically Disadvantaged,sy22_Nonprofit Organization,sy22_Charter Type,sy22_School System Roll Up Type,sy22_2021 % Mastery+ Grades 3-8,sy22_2022 % Mastery+ Grades 3-8,sy22_2021-2022 % Mastery+ Change Grades 3-8,sy22_2021 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History & Biology,sy22_2022 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History & Biology,sy22_2021-2022 % Mastery+ Change Grades 3-8 & English I-II & Algebra I & Geometry & US History & Biology,_merge
4,036189,Homer A. Plessy Community School,397.00,0.48,0.52,2.00,7.00,177.00,48.00,0.00,118.00,45.00,279.00,0.92,0.08,0.00,0.00,39.00,50.00,42.00,46.00,44.00,54.00,53.00,40.00,29.00,0.00,0.00,0.00,0.00,0.00,0.00,0.78,Citizens' Committee for Education,Type 1,R36,19,28,9,19,28,9,036189,Homer A. Plessy Community School,753,0.48,0.52,3,11,395,75,0,205,64,548,0.95,0.05,0,3,51,77,80,83,74,60,77,76,85,87,0,0,0,0,0,0,0.73,Citizens' Committee for Education,Type 1,R36,27,20,-7,27,20,-7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [137]:
sy21switchers = sy21to22_switchers.filter(regex='^sy19|^sy21', axis=1).dropna(how='all').sort_values(by='sy21_Site Name_x')

In [138]:
sy21switchers.shape

(2, 85)

In [139]:
sy22switchers = sy21to22_switchers.filter(regex='^sy22', axis=1).dropna(how='all').sort_values(by='sy22_Site Name_x')

In [140]:
sy22switchers.shape

(2, 43)

In [141]:
sy21_school_name_list = sy21switchers['sy21_Site Name_x'].tolist()

In [142]:
sy21_school_name_list

['Homer A. Plessy Community School', 'Rooted School']

In [143]:
sy22_school_name_list = sy22switchers['sy22_Site Name_x'].tolist()

In [144]:
sy22_school_name_list

['Homer Plessy Community School', 'Rooted School']

In [145]:
sy21switchers = sy21switchers.replace('Homer A. Plessy Community School', 'Homer Plessy Community School')


In [146]:
switchers_19to22final = sy21switchers.merge(sy22switchers, how='inner', left_on='sy21_Site Name_x', right_on ='sy22_Site Name_x')

In [147]:
switchers_19to22final.shape

(2, 128)

In [148]:
switchers_19to22final

,sy19_SiteCd,sy19_SiteName,sy19_Total Students,sy19_%Female,sy19_%Male,sy19_AmInd,sy19_Asian,sy19_Black,sy19_Hispanic,sy19_HawPI,sy19_White,sy19_Multiple,sy19_Minority,sy19_%Fully-EP,sy19_%LEP,sy19_Infants SpEd,sy19_PreSchool SpEd,sy19_PreK,sy19_Kindergarten,sy19_Grade1,sy19_Grade2,sy19_Grade3,sy19_Grade4,sy19_Grade5,sy19_Grade6,sy19_Grade7,sy19_Grade8,sy19_Grade9,sy19_GradeT9,sy19_Grade10,sy19_Grade11,sy19_Grade12,sy19_ED%,sy19_Nonprofit,sy19_Charter Type,sy19_RollUpType,sy19_2018 % Mastery+ Grades 3-8,sy19_2019 % Mastery+ Grades 3-8,sy19_2018-2019 % Mastery+ Change Grades 3-8,sy19_2018 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History,sy19_2019 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History,sy19_2018-2019 % Mastery+ Change Grades 3-8 & English I-II & Algebra I & Geometry & US History,sy21_SIS Submit Site Code,sy21_Site Name_x,sy21_Total Enrollment,sy21_% Female,sy21_% Male,sy21_American Indian,sy21_Asian,sy21_Black,sy21_Hispanic,sy21_Hawaiian/Pacific Islander,sy21_White,sy21_Multiple Races (Non-Hispanic),sy21_Minority,sy21_% Fully English Proficient,sy21_% Limited English Proficient,sy21_Infants (Sp Ed),sy21_Pre-School (Sp Ed),sy21_Pre-K (Reg Ed),sy21_Kindergarten,sy21_Grade 1,sy21_Grade 2,sy21_Grade 3,sy21_Grade 4,sy21_Grade 5,sy21_Grade 6,sy21_Grade 7,sy21_Grade 8,sy21_Grade 9,sy21_Grade T9,sy21_Grade 10,sy21_Grade 11,sy21_Grade 12,sy21_Extension Academy,sy21_% Economically Disadvantaged,sy21_Nonprofit Organization,sy21_Charter Type,sy21_School System Roll Up Type,sy21_2019 % Mastery+ Grades 3-8,sy21_2021 % Mastery+ Grades 3-8,sy21_2019-2021 % Mastery+ Change Grades 3-8,sy21_2019 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History & Biology,sy21_2021 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History & Biology,sy21_2019-2021 % Mastery+ Change Grades 3-8 & English I-II & Algebra I & Geometry & US History & Biology,sy22_SIS Submit Site Code,sy22_Site Name_x,sy22_Total Enrollment,sy22_% Female,sy22_% Male,sy22_American Indian,sy22_Asian,sy22_Black,sy22_Hispanic,sy22_Hawaiian/Pacific Islander,sy22_White,sy22_Multiple Races (Non-Hispanic),sy22_Minority,sy22_% Fully English Proficient,sy22_% Limited English Proficient,sy22_Infants (Sp Ed),sy22_Pre-School (Sp Ed),sy22_Pre-K (Reg Ed),sy22_Kindergarten,sy22_Grade 1,sy22_Grade 2,sy22_Grade 3,sy22_Grade 4,sy22_Grade 5,sy22_Grade 6,sy22_Grade 7,sy22_Grade 8,sy22_Grade 9,sy22_Grade T9,sy22_Grade 10,sy22_Grade 11,sy22_Grade 12,sy22_Extension Academy,sy22_% Economically Disadvantaged,sy22_Nonprofit Organization,sy22_Charter Type,sy22_School System Roll Up Type,sy22_2021 % Mastery+ Grades 3-8,sy22_2022 % Mastery+ Grades 3-8,sy22_2021-2022 % Mastery+ Change Grades 3-8,sy22_2021 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History & Biology,sy22_2022 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History & Biology,sy22_2021-2022 % Mastery+ Change Grades 3-8 & English I-II & Algebra I & Geometry & US History & Biology
0,036189,Homer Plessy Community School,397.00,0.48,0.52,2.00,7.00,177.00,48.00,0.00,118.00,45.00,279.00,0.92,0.08,0.00,0.00,39.00,50.00,42.00,46.00,44.00,54.00,53.00,40.00,29.00,0.00,0.00,0.00,0.00,0.00,0.00,0.78,Citizens' Committee for Education,Type 1,R36,19,28,9,19,28,9,036189,Homer Plessy Community School,753,0.48,0.52,3,11,395,75,0,205,64,548,0.95,0.05,0,3,51,77,80,83,74,60,77,76,85,87,0,0,0,0,0,0,0.73,Citizens' Committee for Education,Type 1,R36,27,20,-7,27,20,-7,3C6001,Homer Plessy Community School,720,0.45,0.55,6,10,342,85,2,211,64,509,0.94,0.06,0,2,49,74,85,69,86,71,68,69,75,72,0,0,0,0,0,0,0.74,NaN,Type 1B,R36,NR,23,NR,NR,24,NR
1,036198,Rooted School,97.00,0.30,0.70,0.00,1.00,84.00,7.00,0.00,5.00,0.00,92.00,0.96,0.04,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,56.00,0.00,41.00,0.00,0.00,0.81,Rooted School,Type 1,R36,NR,NR,NR,37,29,-8,036198,Rooted School,188,0.36,0.64,0,1,155,21,0,11,0,177,0.94,0.06,0,0,0,0,0,0,0,0,0,0,0,0,38

In [149]:
#Replacing rows in 19 to 22 table with correct rows for schools that have switched codes 
#First need to drop the duplicate rows for both years
sy19to22_updated = sy19to22[~sy19to22['sy21_SIS Submit Site Code'].isin(switchers_19to22final['sy21_SIS Submit Site Code'])]

In [150]:
sy19to22.shape

(90, 129)

In [151]:
sy19to22_updated.shape

(88, 129)

In [152]:
sy19to22_updated2 = sy19to22_updated[~sy19to22_updated['sy22_SIS Submit Site Code'].isin(switchers_19to22final['sy22_SIS Submit Site Code'])]

In [153]:
sy19to22_updated2.shape

(86, 129)

In [154]:
#Now need to add the correct rows back in 
sy19to22 = sy19to22_updated2.append(switchers_19to22final)

In [155]:
sy19to22.shape
#Confirmed that rows for the schools that have switched codes are added back in 

(88, 129)

In [156]:
sy19to22.columns.tolist()

['sy19_SiteCd',
 'sy19_SiteName',
 'sy19_Total Students',
 'sy19_%Female',
 'sy19_%Male',
 'sy19_AmInd',
 'sy19_Asian',
 'sy19_Black',
 'sy19_Hispanic',
 'sy19_HawPI',
 'sy19_White',
 'sy19_Multiple',
 'sy19_Minority',
 'sy19_%Fully-EP',
 'sy19_%LEP',
 'sy19_Infants SpEd',
 'sy19_PreSchool SpEd',
 'sy19_PreK',
 'sy19_Kindergarten',
 'sy19_Grade1',
 'sy19_Grade2',
 'sy19_Grade3',
 'sy19_Grade4',
 'sy19_Grade5',
 'sy19_Grade6',
 'sy19_Grade7',
 'sy19_Grade8',
 'sy19_Grade9',
 'sy19_GradeT9',
 'sy19_Grade10',
 'sy19_Grade11',
 'sy19_Grade12',
 'sy19_ED%',
 'sy19_Nonprofit',
 'sy19_Charter Type',
 'sy19_RollUpType',
 'sy19_2018 % Mastery+ Grades 3-8',
 'sy19_2019 % Mastery+ Grades 3-8',
 'sy19_2018-2019 % Mastery+ Change Grades 3-8',
 'sy19_2018 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History',
 'sy19_2019 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History',
 'sy19_2018-2019 % Mastery+ Change Grades 3-8 & English I-II & Algebra I & Geometry & 

In [157]:
#Making sure all the most recent site codes and school names are in the 2022 columns 
sy19to22['sy21_Site Name_x'].fillna(sy19to22['sy19_SiteName'], inplace=True)
sy19to22['sy22_Site Name_x'].fillna(sy19to22['sy21_Site Name_x'], inplace=True)
sy19to22['sy22_Site Name_x'].fillna(sy19to22['sy21_Site Name_x'], inplace=True)
sy19to22['sy22_Site Name_x'].fillna(sy19to22['sy21_Site Name_x'], inplace=True)
sy19to22['sy21_SIS Submit Site Code'].fillna(sy19to22['sy19_SiteCd'], inplace=True)
sy19to22['sy22_SIS Submit Site Code'].fillna(sy19to22['sy21_SIS Submit Site Code'], inplace=True)

In [158]:
#Now that nulls are filled in, I can drop the 2019 and 2021 site code/name columns 
cols_to_move = ['2022_School_Code', 'School_Name']
sy19to22_final = sy19to22.drop(columns=['sy19_SiteName', 'sy21_Site Name_x','sy19_SiteCd','sy21_SIS Submit Site Code'
                                    ,'_merge'], axis=1).rename(columns={'sy22_SIS Submit Site Code':'2022_School_Code',
                                                                       'sy22_Site Name_x':'School_Name'}).reset_index(drop=True)

sy19to22_final = sy19to22_final[cols_to_move + [col for col in sy19to22_final if col not in cols_to_move]]



### IV. Imputation and Final Cleaning Steps

In [159]:
#Moving on to cleaning and inspecting the numerical columns - currently categorized as object columns
#Have to get rid of all the special characters 

#Fills nulls and NRs in dataframe 
sy19to22_final = sy19to22_final.fillna(0)

#Fill NRs with 0 
sy19to22_final = sy19to22_final.replace('NR',0)

#Replace < 5
sy19to22_final = sy19to22_final.replace('< 5', 5)

#Replace ≤ 1 
sy19to22_final = sy19to22_final.replace ('≤ 1', 1)

#Replace >-5 
sy19to22_final = sy19to22_final.replace('>-5', -5)

#Replace > -5 
sy19to22_final = sy19to22_final.replace('> -5',-5)

#Replace < 10
sy19to22_final = sy19to22_final.replace('< 10', 10)

#Replace > -10
sy19to22_final = sy19to22_final.replace('> -10', -10)


In [160]:
object_cols = sy19to22_final.select_dtypes(include=['object'])

In [161]:
cols_to_transform = object_cols.drop(columns = ['2022_School_Code','School_Name','sy19_Nonprofit','sy19_Charter Type',
                                                'sy19_RollUpType','sy21_Nonprofit Organization','sy21_Charter Type'
                                               ,'sy21_School System Roll Up Type','sy22_Nonprofit Organization',
                                                'sy22_Charter Type','sy22_School System Roll Up Type'], axis=1)

In [162]:
for col in cols_to_transform:
    print(cols_to_transform[col].unique())

['   30' 0 5 '   40' '   15' '   16' '   35' '   12' '   13' '    7'
 '   14' '   22' '   18' '   24' '   27' '   25' '   20' '   23' '   19'
 '   50' '   39' '   87' '   86' '   43' '   21' '   17']
['   17' 0 5 '   15' '   16' '    6' '    9' '    8' '    7' '   24'
 '   10' '   12' '    5' '   14' '   13' '   11' '   40' '   22' '   82'
 '   75' '   33' '   56' '   20']
['  -13' 0 5 '  -15' '  -24' 10 '   -6' '   -8' '  -11' '   -2' '   -4'
 '   -5' -5 '  -10' '   -9' '   -3' '  -17' '   -1' '   -7']
['   30' '    7' 5 '   31' 0 '   40' '    8' '   15' '   16' '   35'
 '   12' '   13' '   14' '   24' '   22' '   21' '   10' '   18' '   27'
 '   29' '   25' '   20' '   23' '   19' '   50' '   95' '   88' '   87'
 '   17' '   43' '   39' '   26']
['   17' 5 '   15' 0 '   16' '    7' '    5' '    9' '    8' '   23'
 '   10' '   12' '   13' '   14' '   19' '   11' '    6' '   39' '   89'
 '   22' '   18' '   82' '   76' '   33' '   55' '   20']
['  -13' -10 5 '  -16' 0 '  -24' '   -1' '

In [163]:
cols = object_cols.drop(columns = ['2022_School_Code','School_Name','sy19_Nonprofit','sy19_Charter Type',
                                                'sy19_RollUpType','sy21_Nonprofit Organization','sy21_Charter Type'
                                               ,'sy21_School System Roll Up Type','sy22_Nonprofit Organization',
                                                'sy22_Charter Type','sy22_School System Roll Up Type'], axis=1).columns.tolist()

In [164]:
sy19to22_final[cols] = sy19to22_final[cols].apply(pd.to_numeric, errors='coerce')

In [165]:
sy19to22_final.dtypes

2022_School_Code                                                                                             object
School_Name                                                                                                  object
sy19_Total Students                                                                                         float64
sy19_%Female                                                                                                float64
sy19_%Male                                                                                                  float64
sy19_AmInd                                                                                                  float64
sy19_Asian                                                                                                  float64
sy19_Black                                                                                                  float64
sy19_Hispanic                                                           

In [166]:
#Dropping 3 alternative schools from final table as per task instructions
#Drop 360001, 360002, and 036132 from analysis from final table

sy19to22_final = sy19to22_final.loc[~((sy19to22_final['2022_School_Code'] == '360001') | (sy19to22_final['2022_School_Code'] == '360002') | 
                               (sy19to22_final['2022_School_Code'] == '036132'))]

In [167]:
sy19to22_final.shape
#Confirmed that 3 alternative schools are dropped 

(85, 124)

In [168]:
cols_list = sy19to22_final.columns.tolist()
cols_list

['2022_School_Code',
 'School_Name',
 'sy19_Total Students',
 'sy19_%Female',
 'sy19_%Male',
 'sy19_AmInd',
 'sy19_Asian',
 'sy19_Black',
 'sy19_Hispanic',
 'sy19_HawPI',
 'sy19_White',
 'sy19_Multiple',
 'sy19_Minority',
 'sy19_%Fully-EP',
 'sy19_%LEP',
 'sy19_Infants SpEd',
 'sy19_PreSchool SpEd',
 'sy19_PreK',
 'sy19_Kindergarten',
 'sy19_Grade1',
 'sy19_Grade2',
 'sy19_Grade3',
 'sy19_Grade4',
 'sy19_Grade5',
 'sy19_Grade6',
 'sy19_Grade7',
 'sy19_Grade8',
 'sy19_Grade9',
 'sy19_GradeT9',
 'sy19_Grade10',
 'sy19_Grade11',
 'sy19_Grade12',
 'sy19_ED%',
 'sy19_Nonprofit',
 'sy19_Charter Type',
 'sy19_RollUpType',
 'sy19_2018 % Mastery+ Grades 3-8',
 'sy19_2019 % Mastery+ Grades 3-8',
 'sy19_2018-2019 % Mastery+ Change Grades 3-8',
 'sy19_2018 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History',
 'sy19_2019 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History',
 'sy19_2018-2019 % Mastery+ Change Grades 3-8 & English I-II & Algebra I & Geometry

In [169]:
#Make 3-8 enrollment totals for all 3 years 
sy19to22_final['sy19_gr3_8_pop'] = sy19to22_final['sy19_Grade3']+sy19to22_final['sy19_Grade4'] +sy19to22_final['sy19_Grade5']+sy19to22_final['sy19_Grade6']+sy19to22_final['sy19_Grade7']+sy19to22_final['sy19_Grade8']
sy19to22_final['sy21_gr3_8_pop'] = sy19to22_final['sy21_Grade 3']+sy19to22_final['sy21_Grade 4'] +sy19to22_final['sy21_Grade 5']+sy19to22_final['sy21_Grade 6']+sy19to22_final['sy21_Grade 7']+sy19to22_final['sy21_Grade 8']
sy19to22_final['sy22_gr3_8_pop'] = sy19to22_final['sy22_Grade 3']+sy19to22_final['sy22_Grade 4'] +sy19to22_final['sy22_Grade 5']+sy19to22_final['sy22_Grade 6']+sy19to22_final['sy22_Grade 7']+sy19to22_final['sy22_Grade 8']

In [170]:
#Multiply percent columns by 100 so everything is in scale
to_multiply = ['sy19_%Female','sy19_%Male','sy19_%Fully-EP','sy19_%LEP','sy19_ED%',
                            'sy21_% Female','sy21_% Male','sy21_% Fully English Proficient','sy21_% Limited English Proficient',
                            'sy21_% Economically Disadvantaged','sy22_% Female','sy22_% Male','sy22_% Fully English Proficient',
                            'sy22_% Limited English Proficient','sy22_% Economically Disadvantaged']

#sy19to22_final = sy19to22_final['']
for column in sy19to22_final.columns: 
    if column in to_multiply: 
        sy19to22_final[column] = sy19to22_final[column] * 100

In [171]:
#Renaming some columns for brevity
sy19to22_final = sy19to22_final.rename(columns={'sy19_2019 % Mastery+ Grades 3-8':'SY19_%Mastery_3-8',
                                                'sy21_2021 % Mastery+ Grades 3-8': 'SY21_%Mastery_3-8',
                                               'sy22_2022 % Mastery+ Grades 3-8': 'SY22_%Mastery_3-8',
                                               'sy21_% Economically Disadvantaged':'sy21_ed%',
                                               'sy22_% Economically Disadvantaged': 'sy22_ed%'})

In [177]:
#Add average Mastery rate for all three years 
sy19to22_final['3year_mastery_avg_3-8'] = sy19to22_final[['SY19_%Mastery_3-8','SY21_%Mastery_3-8','SY22_%Mastery_3-8']].mean(axis=1)

<ipython-input-177-11e80c15f23f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sy19to22_final['3year_mastery_avg'] = sy19to22_final[['SY19_%Mastery_3-8','SY21_%Mastery_3-8','SY22_%Mastery_3-8']].mean(axis=1)


In [180]:
#Bring 3-8 Mastery rates to front of dataframe
cols_to_move = ['2022_School_Code', 'School_Name','SY19_%Mastery_3-8','sy19_ED%','sy19_gr3_8_pop',
                'SY21_%Mastery_3-8','sy21_ed%','sy21_gr3_8_pop',
               'SY22_%Mastery_3-8','sy22_ed%','sy22_gr3_8_pop','3year_mastery_avg']

sy19to22_final = sy19to22_final[cols_to_move + [col for col in sy19to22_final if col not in cols_to_move]]

In [181]:
sy19to22_final.head()

,2022_School_Code,School_Name,SY19_%Mastery_3-8,sy19_ED%,sy19_gr3_8_pop,SY21_%Mastery_3-8,sy21_ed%,sy21_gr3_8_pop,SY22_%Mastery_3-8,sy22_ed%,sy22_gr3_8_pop,3year_mastery_avg,sy19_Total Students,sy19_%Female,sy19_%Male,sy19_AmInd,sy19_Asian,sy19_Black,sy19_Hispanic,sy19_HawPI,sy19_White,sy19_Multiple,sy19_Minority,sy19_%Fully-EP,sy19_%LEP,sy19_Infants SpEd,sy19_PreSchool SpEd,sy19_PreK,sy19_Kindergarten,sy19_Grade1,sy19_Grade2,sy19_Grade3,sy19_Grade4,sy19_Grade5,sy19_Grade6,sy19_Grade7,sy19_Grade8,sy19_Grade9,sy19_GradeT9,sy19_Grade10,sy19_Grade11,sy19_Grade12,sy19_Nonprofit,sy19_Charter Type,sy19_RollUpType,sy19_2018 % Mastery+ Grades 3-8,sy19_2018-2019 % Mastery+ Change Grades 3-8,sy19_2018 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History,sy19_2019 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History,sy19_2018-2019 % Mastery+ Change Grades 3-8 & English I-II & Algebra I & Geometry & US History,sy21_Total Enrollment,sy21_% Female,sy21_% Male,sy21_American Indian,sy21_Asian,sy21_Black,sy21_Hispanic,sy21_Hawaiian/Pacific Islander,sy21_White,sy21_Multiple Races (Non-Hispanic),sy21_Minority,sy21_% Fully English Proficient,sy21_% Limited English Proficient,sy21_Infants (Sp Ed),sy21_Pre-School (Sp Ed),sy21_Pre-K (Reg Ed),sy21_Kindergarten,sy21_Grade 1,sy21_Grade 2,sy21_Grade 3,sy21_Grade 4,sy21_Grade 5,sy21_Grade 6,sy21_Grade 7,sy21_Grade 8,sy21_Grade 9,sy21_Grade T9,sy21_Grade 10,sy21_Grade 11,sy21_Grade 12,sy21_Extension Academy,sy21_Nonprofit Organization,sy21_Charter Type,sy21_School System Roll Up Type,sy21_2019 % Mastery+ Grades 3-8,sy21_2019-2021 % Mastery+ Change Grades 3-8,sy21_2019 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History & Biology,sy21_2021 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History & Biology,sy21_2019-2021 % Mastery+ Change Grades 3-8 & English I-II & Algebra I & Geometry & US History & Biology,sy22_Total Enrollment,sy22_% Female,sy22_% Male,sy22_American Indian,sy22_Asian,sy22_Black,sy22_Hispanic,sy22_Hawaiian/Pacific Islander,sy22_White,sy22_Multiple Races (Non-Hispanic),sy22_Minority,sy22_% Fully English Proficient,sy22_% Limited English Proficient,sy22_Infants (Sp Ed),sy22_Pre-School (Sp Ed),sy22_Pre-K (Reg Ed),sy22_Kindergarten,sy22_Grade 1,sy22_Grade 2,sy22_Grade 3,sy22_Grade 4,sy22_Grade 5,sy22_Grade 6,sy22_Grade 7,sy22_Grade 8,sy22_Grade 9,sy22_Grade T9,sy22_Grade 10,sy22_Grade 11,sy22_Grade 12,sy22_Extension Academy,sy22_Nonprofit Organization,sy22_Charter Type,sy22_School System Roll Up Type,sy22_2021 % Mastery+ Grades 3-8,sy22_2021-2022 % Mastery+ Change Grades 3-8,sy22_2021 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History & Biology,sy22_2022 % Mastery+ Grades 3-8 & English I-II & Algebra I & Geometry & US History & Biology,sy22_2021-2022 % Mastery+ Change Grades 3-8 & English I-II & Algebra I & Geometry & US History & Biology
0,036011,Mary Bethune Elementary Literature/Technology,33,81.83,412.00,17,85.47,442,20,92.70,443,23.33,688.00,52.18,47.82,0.00,2.00,669.00,15.00,0.00,2.00,0.00,686.00,98.55,1.45,0.00,2.00,38.00,78.00,78.00,80.00,81.00,79.00,82.00,54.00,55.00,61.00,0.00,0.00,0.00,0.00,0.00,0,0,R36,31,2,31,34,3,709,51.62,48.38,0,0,692,17,0,0,0,709,98.87,1.13,0,6,32,76,75,78,78,78,76,78,79,53,0,0,0,0,0,0,"Significant Educators, Inc.",Type 3,R36,30,-13,30,17,-13,699,51.36,48.64,0,0,678,20,1,0,0,699,98.14,1.86,0,8,32,70,74,72,66,70,77,74,78,78,0,0,0,0,0,0,"Significant Educators, Inc.",Type 3,R36,17,3,17,21,4
1,036088,McDonogh #35 College Preparatory School,0,88.76,0.00,0,83.59,0,0,0.00,0,0.00,445.00,44.27,55.73,0.00,1.00,442.00,2.00,0.00,0.00,0.00,445.00,99.55,0.45,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,10.00,0.00,146.00,107.00,182.00,0,0,R36,0,0,9,8,-1,128,44.53,55.47,0,0,127,1,0,0,0,128,99.22,0.78,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,127,0,0,0,R36,0,0,7,5,-10,0,0.00,0.00,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,036161,Benjami

### V. Answering Question 1

In [183]:
sy19_gr3_8_pop = sy19to22_final.loc[sy19to22_final['sy19_gr3_8_pop'] != 0]
print(sy19_gr3_8_pop.shape)

(54, 128)


54 schools served students in grades 3-8 in 2019.

In [184]:
sy21_gr3_8_pop = sy19to22_final.loc[sy19to22_final['sy21_gr3_8_pop'] != 0]
print(sy21_gr3_8_pop.shape)

(50, 128)


50 schools served students in grades 3-8 in 2021.

In [186]:
sy22_gr3_8_pop = sy19to22_final.loc[sy19to22_final['sy22_gr3_8_pop'] != 0]
print(sy22_gr3_8_pop.shape)

(50, 128)


...and 50 schools served students in grades 3-8 in 2022. 

In [190]:
#Saving final table as an Excel spreadsheet 
sy19to22_final.to_excel('final_table.xlsx')

In [191]:
#Creating subset file of just the grade 3-8 data, for visualization 
grade_3_8_table_1 = sy19_gr3_8_pop.append(sy21_gr3_8_pop)

In [193]:
grade_3_8_table_2 = grade_3_8_table_1.append(sy22_gr3_8_pop)
print(grade_3_8_table_2.shape)

(154, 128)


In [194]:
grade_3_8_table_2.to_excel('leap_table.xlsx')

### VI. Answering Question 5

In terms of considerations, caveats, and assumptions, I didn't individually Google all the schools whose school name was also their "School System Name" in the multistat files. I checked Lycee Francais and confirmed it to be a Type 2 charter, so I assumed all the others were Type 2 charters, but did not check each one. 

I would advise anyone using this code to review the "School System Name" lists and make sure that the table does not exclude schools they would like to include - especially if they are looking at schools outside of Orleans Parish. I am also unsure how fluid this Type 2 category is (do schools move between Type 1 and Type 2 from year to year?), so colleagues should exercise caution and double-check the status of the schools they are interested in to ensure that they aren't leaving out relevant data. 

In addition, use the three-year Mastery averages with caution. Right now, the schools who enrolled students for fewer than three years (for example, a school that did not open until 2020) have Mastery averages that are artifically lowered, as there are zeroes for the years in which that school did not enroll students. These zeroes bring down the total average considerably. Given more time, I can come up with solutions for imputing the data with another character and re-doing the calculations for this subset of schools. 

Lastly, I am unsure how to treat schools that have multiple campuses (for example, Hynes). Each campus seems to have its own specific site codes, but more discussion may be needed on if this data for multiple capuses should be averaged together, etc. for accountability purposes. 
